In [1]:
import geopandas as gpd
import pandas as pd
import boto3

In [2]:
bucket_name = 'opera-validation-datasets'
root_folder_name = 'calval-database-dev/'
upload_files_path = './upload/'

In [3]:
filename = '20211011_155455_52_2262_3B_AnalyticMS_SR_8b_clip_Supervised_MD_reclass.tif'
logfile = None
training_data = None
source_image_name = '20211011_155455_52_2262'
calc_type = 'Reclassification'
final = True
fitness = 0
public = True

In [4]:
session = boto3.session.Session(profile_name='saml-pub')
s3 = session.resource('s3')
s3_client = session.client('s3')

In [5]:
collocation = gpd.read_file('Collocation_HLS_Planet_2021.geojson')
search = collocation[(collocation.pl_ID == source_image_name)].sort_values('hls_pl_AOI_intersect',ascending=False)
search = search.iloc[[0]]
site = search.chip_ID.iloc[0]
new_image_calc = search[['pl_ID','geometry']].rename(columns={'pl_ID':'image_name'})


In [6]:
new_image_calc['image_calc_name'] = filename[:-4]
new_image_calc['calc_type'] = calc_type
new_image_calc['final'] = final
new_image_calc['fitness'] = fitness
new_image_calc['public'] = public


In [7]:
s3_folder_path = root_folder_name+'data/'+site+'/'+source_image_name+'/'+filename[:-4]+'/'
s3_link_image = s3_folder_path+filename
response = s3_client.upload_file(upload_files_path+filename,bucket_name,s3_link_image)
if logfile != None:
    s3_link_logfile = s3_folder_path+logfile
    response = s3_client.upload_file(upload_files_path+logfile,bucket_name,s3_link_logfile)
else:
    s3_link_logfile = None
if training_data != None:
    s3_link_training = s3_folder_path+training_data
    response = s3_client.upload_file(upload_files_path+training_data,bucket_name,s3_link_training)
else:
    s3_link_training = None

In [8]:
new_image_calc['s3_link_image_calc'] = s3_link_image
new_image_calc['s3_link_logfile'] = s3_link_logfile
new_image_calc['s3_link_training'] = s3_link_training

In [9]:
obj = s3.Object(bucket_name,root_folder_name+'image_calc.geojson')
image_calc_data = obj.get()['Body']
image_calc_gdf = gpd.read_file(image_calc_data)
image_calc_gdf.head()

,calc_type,final,fitness,image_calc_name,image_name,public,s3_link_image_calc,s3_link_logfile,s3_link_training,geometry
0,Classification,False,0,20211011_155455_52_2262_3B_AnalyticMS_SR_8b_cl...,20211011_155455_52_2262,True,calval-database-dev/data/3_10/20211011_155455_...,calval-database-dev/data/3_10/20211011_155455_...,calval-database-dev/data/3_10/20211011_155455_...,"POLYGON ((-90.50851 16.35841, -90.54697 16.176..."


In [10]:
image_calc_upd_gdf = image_calc_gdf.append(new_image_calc,ignore_index=True)
image_calc_upd_gdf.head()

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,calc_type,final,fitness,geometry,image_calc_name,image_name,public,s3_link_image_calc,s3_link_logfile,s3_link_training
0,Classification,False,0,"POLYGON ((-90.50851 16.35841, -90.54697 16.176...",20211011_155455_52_2262_3B_AnalyticMS_SR_8b_cl...,20211011_155455_52_2262,True,calval-database-dev/data/3_10/20211011_155455_...,calval-database-dev/data/3_10/20211011_155455_...,calval-database-dev/data/3_10/20211011_155455_...
1,Reclassification,True,0,"POLYGON ((-90.50851 16.35841, -90.54697 16.176...",20211011_155455_52_2262_3B_AnalyticMS_SR_8b_cl...,20211011_155455_52_2262,True,calval-database-dev/data/3_10/20211011_155455_...,None,None


In [11]:
image_calc_upd_bytes = bytes(image_calc_upd_gdf.to_json(drop_id=True).encode('UTF-8'))
s3object = s3.Object(bucket_name,root_folder_name+'image_calc.geojson')
s3object.put(Body=image_calc_upd_bytes)

{'ResponseMetadata': {'RequestId': 'VXETV2V0H4X5596R',
  'HostId': 'vqQqLTRGImP/L+GlfgfxhDH5gDxa4H+XFTpDZX1xEO6uYBYYBg3wpgrpmE+4iC2Z4ZC6bpPZ/NY=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'vqQqLTRGImP/L+GlfgfxhDH5gDxa4H+XFTpDZX1xEO6uYBYYBg3wpgrpmE+4iC2Z4ZC6bpPZ/NY=',
   'x-amz-request-id': 'VXETV2V0H4X5596R',
   'date': 'Tue, 21 Jun 2022 17:21:31 GMT',
   'etag': '"50e9b1c0255494bca300b671cb69963b"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"50e9b1c0255494bca300b671cb69963b"'}

In [12]:
image_calc_data = obj.get()['Body']
image_calc_gdf = gpd.read_file(image_calc_data)
image_calc_gdf.head()

,calc_type,final,fitness,image_calc_name,image_name,public,s3_link_image_calc,s3_link_logfile,s3_link_training,geometry
0,Classification,False,0,20211011_155455_52_2262_3B_AnalyticMS_SR_8b_cl...,20211011_155455_52_2262,True,calval-database-dev/data/3_10/20211011_155455_...,calval-database-dev/data/3_10/20211011_155455_...,calval-database-dev/data/3_10/20211011_155455_...,"POLYGON ((-90.50851 16.35841, -90.54697 16.176..."
1,Reclassification,True,0,20211011_155455_52_2262_3B_AnalyticMS_SR_8b_cl...,20211011_155455_52_2262,True,calval-database-dev/data/3_10/20211011_155455_...,None,None,"POLYGON ((-90.50851 16.35841, -90.54697 16.176..."
